In [8]:
import librosa
import pyAudioAnalysis as pyaudio
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import pandas as pd
import os

In [2]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures

In [3]:
features = []
existing = []
missing = []
prev_features = None

In [4]:
if os.path.exists("../data/features.csv"):
    prev_features = pd.read_csv("../data/features.csv")
    prev_features.dropna(how = 'any')
    prev_features['song'] = prev_features['title'] + " - " + prev_features['artist']
    existing = prev_features['song'].tolist()

In [5]:
for song in os.scandir("."):
    if song.path.endswith(".mp3") and song.is_file():
        # skip ./ in the path to the mp3 file
        file_name = song.path[2:]
        song_name = file_name[:len(file_name) - 4]
        if song_name not in existing:
            # add it to the missing list so we can use pyAudioAnalysis on it
            missing.append(file_name)
            
            # get the tempo of the song
            waveform, samp_rate = librosa.load(file_name)
            tempo, beat_frames = librosa.beat.beat_track(waveform, samp_rate)

            # get the chroma number of the song
            beat_times = librosa.frames_to_time(beat_frames, samp_rate)
            y_harmonic, y_percussive = librosa.effects.hpss(waveform)
            chromagram = librosa.feature.chroma_cqt(y_harmonic, samp_rate)
            beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate = np.median)
            # make beat chroma into a DataFrame and calculate the diff
            chroma_df = pd.DataFrame(beat_chroma)
            diff_values = chroma_df.diff()
            diff_mean = diff_values.mean(axis = 0, skipna = True)
            chroma_num = sum(diff_mean) / len(diff_mean)

            # add song, tempo and chroma number to the features list
            print ([song_name, tempo, chroma_num])
            features.append([song_name, tempo, chroma_num])

/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


["baby don't cry - exo", 69.83741554054055, -0.01112832059264044]


In [6]:
features_df = pd.DataFrame(features, columns = ['song', 'tempo', 'chroma_number'])
features_df = features_df.sort_values(by = ['song'])

In [7]:
# move all the mp3 files that need to have features extracted into some folder
with open('missing.txt', 'w+') as f:
    for song in missing:
        f.write("%s\n" % song)
# use command line here
'''
mkdir missing
cat missing.txt | while read line
do
    cp $line missing
done
'''
# change the path in directory_feature_extraction

'\nmkdir missing\ncat missing.txt | while read line\ndo\n    cp $line missing\ndone\n'

In [3]:
mid_term_window = 1
mid_term_step = 1
short_term_window = 0.05
short_term_step = 0.05
# use compute_beat = True if we want the extra beat features
pyaudio_feat, files, feat_names = MidTermFeatures.directory_feature_extraction(".", 
                                                                               mid_term_window, 
                                                                               mid_term_step, 
                                                                               short_term_window, 
                                                                               short_term_step,
                                                                               False)

Analyzing file 1 of 1: trial/traveler of the magic borders - takanashi yasuharu.mp3
Feature extraction complexity ratio: 43.9 x realtime


In [25]:
# neural network features
features_df.reset_index(drop = True, inplace = True)
if len(features_df) == 1:
    nn_features_df = pd.DataFrame([pyaudio_feat], columns = feat_names)
else:
    nn_features_df = pd.DataFrame(pyaudio_feat, columns = feat_names)
nn_features_df['song'] = features_df['song']

In [26]:
zcr_ind = feat_names.index('zcr_mean')
ee_ind = feat_names.index('energy_entropy_mean')
spc_ind = feat_names.index('spectral_centroid_mean')

features_df['zero_crossing_rate'] = nn_features_df.iloc[:, zcr_ind]
features_df['energy_entropy'] = nn_features_df.iloc[:, ee_ind]
features_df['spectral_centroid'] = nn_features_df.iloc[:, spc_ind]

In [27]:
features_df

,song,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid
0,baby don't cry - exo,69.837416,-0.011128,0.053817,3.185847,0.136686


In [62]:
moods = pd.read_csv("../data/mood_data.csv")
moods

,title,artist,primary,secondary
0,who says,selena gomez,2,3
1,don't stop,5 seconds of summer,3,2
2,do you hear what i hear,carrie underwood,4,2
3,"alliance force, assemble",takanashi yasuharu,1,6
4,ni pa bu pa shi qu wo,liu zeng tong,2,4
...,...,...,...,...
195,nine four two zero,mai xiao er,2,4
196,zui mei hun li,bai xiao bai,2,4
197,jiu shi xi huan ni,li meng yin,2,6
198,xiao xing xing,wang su long,2,4


In [63]:
# merging is incorrect, does not work
moods['song'] = moods['title'] + " - " + moods['artist']
moods = moods.sort_values(by = ['song'])
moods.reset_index(drop = True, inplace = True)

new_features = features_df.merge(moods, on = "song", how = "left")
new_features = new_features.drop(columns = ['song'])
new_features

,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid,title,artist,primary,secondary
0,69.837416,-0.011128,0.053817,3.185847,0.136686,"baby, don't cry",exo,2,4


In [15]:
if not existing:
    new_features = new_features[['title', 'artist', 'tempo', 'chroma_number', 
                                 'zero_crossing_rate', 'energy_entropy', 
                                 'spectral_centroid', 'primary', 'secondary']]
    new_features.to_csv("../data/features.csv", header = True, index = False)
    
    nn_features_df.to_csv("../data/nn_features.csv", header = True, index = False)
    
    feat_names_df = pd.DataFrame(feat_names)
    feat_names_df.to_csv("../data/nn_feature_names.csv", header = True, index = False)
else:
    prev_features = prev_features.drop(columns = ['song'])
    all_features = pd.concat([prev_features, new_features], ignore_index = True)
    all_features = all_features[['title', 'artist', 'tempo', 'chroma_number', 
                                 'zero_crossing_rate', 'energy_entropy', 
                                 'spectral_centroid', 'primary', 'secondary']]
    all_features.to_csv("../data/features.csv", header = True, index = False)
    
    prev_nn = pd.read_csv("../data/nn_features.csv")
    all_nn = pd.concat([prev_nn, nn_features_df], ignore_index = True)
    all_nn.to_csv("../data/nn_features.csv", header = True, index = False)

In [ ]:
# make sure to delete the directory missing and the text file missing.txt
'''
rm -r missing
rm missing.txt
'''

In [28]:
# extract additional neural network features
# ! check ffmpeg at some point !
existing_raw = []
new_raw_dict = {}

In [29]:
# check existing raw features
if os.path.exists("../data/raw_features.csv"):
    prev_raw = pd.read_csv("../data/raw_features.csv")
    prev_raw.dropna(how = 'any')
    prev_raw['song'] = prev_raw['title'] + " - " + prev_raw['artist']
    existing_raw = prev_raw['song'].tolist()

In [31]:
for song in os.scandir("."):
    if song.path.endswith(".mp3") and song.is_file():
        # skip ./ in the path to the mp3 file
        file_name = song.path[2:]
        song_name = file_name[:len(file_name) - 4]
        if song_name not in existing_raw:
            sound = AudioSegment.from_mp3(file_name)
            sound = sound.set_channels(1)
            sound.export("temp.wav", format = "wav")
            # does not work for mp3s that are too short or have an extremely low sample rate
            for block in sf.blocks("temp.wav", blocksize = 500000, start = 500000):
                new_raw_dict[song_name] = block
                break
if os.path.exists("temp.wav"):
    os.remove("temp.wav")
else:
    print ("No temporary wav file found, extraction failed.")

In [34]:
new_raw = pd.DataFrame(new_raw_dict)
new_raw = new_raw.transpose()
new_raw.index.name = 'song'
new_raw.reset_index(inplace = True)

In [35]:
new_raw

,song,0,1,2,3,4,5,6,7,8,...,499990,499991,499992,499993,499994,499995,499996,499997,499998,499999
0,"zhe shi ai - henry, donghae",0.000061,0.000061,-0.000061,-0.000153,-0.000122,-0.000153,-0.000092,0.000000,0.000031,...,0.033447,0.036987,0.040314,0.043274,0.045807,0.047699,0.049164,0.050629,0.052307,0.054321
1,ai no katachi - tachibana keita,-0.455383,-0.478729,-0.497406,-0.476715,-0.410675,-0.352478,-0.308044,-0.311218,-0.342316,...,0.229370,0.205353,0.188599,0.214935,0.451721,0.671875,0.717773,0.711731,0.521362,0.204895
2,a thousand years - christina perri,0.084137,0.084930,0.085846,0.087311,0.090057,0.093475,0.096039,0.097870,0.099182,...,-0.109833,-0.107452,-0.117767,-0.113617,-0.095703,-0.088531,-0.099426,-0.110168,-0.106323,-0.103333
3,red - taylor swift,-0.025879,0.185913,0.097565,-0.144836,-0.173065,0.027557,0.183929,0.143646,-0.006012,...,0.025299,0.039856,0.005127,-0.044830,-0.008942,0.095978,0.155396,0.104614,0.015930,-0.001923
4,"no air - jordin sparks, chris brown",-0.078400,-0.088654,-0.085815,-0.079651,-0.070587,-0.056091,-0.045532,-0.036835,-0.019470,...,-0.023285,-0.027130,-0.030365,-0.028870,-0.030731,-0.037933,-0.041473,-0.040771,-0.039612,-0.035553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,cobra's poison - takanashi yasuharu,-0.069519,-0.065918,-0.059143,-0.015381,0.024872,0.004089,-0.030090,-0.039062,-0.061493,...,0.197998,0.362701,0.491302,0.510620,0.453033,0.336792,0.249847,0.232727,0.201019,0.237396
196,kataomoi - aimer,-0.210785,-0.186676,-0.177826,-0.170044,-0.159332,-0.154785,-0.149750,-0.136719,-0.124664,...,-0.384094,-0.434052,-0.404999,-0.338745,-0.333923,-0.373535,-0.395294,-0.401428,-0.426636,-0.468475
197,crimson lotus - takanashi yasuharu,-0.050049,-0.083710,-0.132568,-0.149597,-0.195892,-0.306030,-0.345428,-0.346191,-0.343811,...,-0.483704,-0.499847,-0.449036,-0.387573,-0.341095,-0.278381,-0.206116,-0.242523,-0.376587,-0.444885
198,polaris memories - sereno,0.027344,0.022430,0.020264,0.020020,0.019196,0.017426,0.016602,0.017914,0.022858,...,0.067902,0.060272,0.049408,0.033966,0.018982,0.008911,0.003784,0.005676,0.016083,0.029968


In [36]:
if not existing_raw:
    new_raw.to_csv("../data/raw_features.csv", header = True, index = False)
else:
    all_raw = pd.concat([prev_raw, new_raw], ignore_index = True)
    all_raw.to_csv("../data/raw_features.csv", header = True, index = False)